# Advanced Querying Mongo

Importing libraries and setting up connection

In [12]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()
list(db.list_collections())

[{'name': 'companies',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('27a8d825-b258-44f8-848c-97792d131793')},
  'idIndex': {'v': 2,
   'key': {'_id': 1},
   'name': '_id_',
   'ns': 'companies.companies'}}]

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [184]:
# Your Code
q1 = {"name":"Babelgum"}
list(db.companies.find(q1, projection={"name"}))

[{'_id': ObjectId('52cdef7c4bab8bd675297da0'), 'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [61]:
# Your Code
q2 = {"number_of_employees": {'$gt':5000}}
(list(db.companies.find(q2, projection={"name","number_of_employees"}))[:20])

[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'number_of_employees': 5299},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9b'),
  'name': 'eBay',
  'number_of_employees': 15000},
 {'_id': ObjectId('52cdef7c4bab8bd675297da2'),
  'name': 'Cisco',
  'number_of_employees': 63000},
 {'_id': ObjectId('52cdef7c4bab8bd675297da3'),
  'name': 'Yahoo!',
  'number_of_employees': 13600},
 {'_id': ObjectId('52cdef7c4bab8bd675297dc4'),
  'name': 'Intel',
  'number_of_employees': 86300},
 {'_id': ObjectId('52cdef7c4bab8bd675297dba'),
  'name': 'Google',
  'number_of_employees': 28000},
 {'_id': ObjectId('52cdef7c4bab8bd675297e49'),
  'name': 'Nintendo',
  'number_of_employees': 5080},
 {'_id': ObjectId('52cdef7c4bab8bd675297e6f'),
  'name': 'Sony',
  'number_of_employees': 180500},
 {'_id': ObjectId('52cdef7c4bab8bd675297e5d'),
  'name': 'Adobe Systems',
  'number_of_employees': 7173},
 {'_id': ObjectId('52cdef7c4bab8bd675297e8e'),
  'name': 'The Walt Disney Company',
  'number_of_

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [183]:
# Your Code
q3 = {"founded_year": {'$gte':2000, '$lte':2005}}
list(db.companies.find(q3, projection={"name", "founded_year"}).limit(5))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8c'),
  'name': 'Zoho',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8f'),
  'name': 'Omnidrive',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'),
  'name': 'Digg',
  'founded_year': 2004},
 {'_id': ObjectId('52cdef7c4bab8bd675297d96'),
  'name': 'Gizmoz',
  'founded_year': 2003}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [79]:
# Your Code:
q4 = {"$and": [{"ipo.valuation_amount":{'$gte':100000000}},{"founded_year":{'$lt':2010}}]}
list(db.companies.find(q4, projection={"name", "ipo"}).limit(5))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d94'),
  'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297de0'),
  'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e0c'),
  'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e81

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [148]:
# Your Code
q5 = {"$and": [{"number_of_employees":{'$lte':1000}},{"founded_year":{'$lt':2005}}]}
list((db.companies.find(q5, {"name":1, "_id":0}, sort=[("number_of_employees",1)]).limit(10)))

[{'name': 'Fox Interactive Media'},
 {'name': 'Skype'},
 {'name': 'Ticketmaster'},
 {'name': 'stylediary'},
 {'name': 'MindTouch'},
 {'name': 'Simpy'},
 {'name': 'Eurekster'},
 {'name': 'Compete'},
 {'name': 'EditGrid'},
 {'name': 'Monster'}]

### 6. All the companies that don't include the `partners` field.

In [152]:
# Your Code
q6 = { "partners": { '$exists': False }}
list(db.companies.find(q6).limit(5))  

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [181]:
# Your Code
q7 = { "category_code": { '$type': "null" }}
list(db.companies.find(q7).limit(5)) 

[{'_id': ObjectId('52cdef7c4bab8bd6752980f6'),
  'name': 'Collective',
  'permalink': 'collective',
  'crunchbase_url': 'http://www.crunchbase.com/company/collective',
  'homepage_url': None,
  'blog_url': None,
  'blog_feed_url': None,
  'twitter_username': None,
  'category_code': None,
  'number_of_employees': None,
  'founded_year': None,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': None,
  'alias_list': None,
  'email_address': None,
  'phone_number': None,
  'description': None,
  'created_at': 'Thu Sep 26 13:15:02 UTC 2013',
  'updated_at': 'Thu Sep 26 13:15:02 UTC 2013',
  'overview': None,
  'image': None,
  'products': [],
  'relationships': [],
  'competitions': [],
  'providerships': [],
  'total_money_raised': '$0',
  'funding_rounds': [],
  'investments': [],
  'acquisition': None,
  'acquisitions': [],
  'offices': [],
  'milestones': [],
  'ipo': N

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [180]:
# Your Code:
q8 = {"number_of_employees":{'$gte':100, '$gte':1000}}
list(db.companies.find(q8, projection={"name", "number_of_employees"}).limit(5))

[{'_id': ObjectId('52cdef7c4bab8bd675297d8c'),
  'name': 'Zoho',
  'number_of_employees': 1600},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'number_of_employees': 5299},
 {'_id': ObjectId('52cdef7c4bab8bd675297d94'),
  'name': 'Twitter',
  'number_of_employees': 1300},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9b'),
  'name': 'eBay',
  'number_of_employees': 15000},
 {'_id': ObjectId('52cdef7c4bab8bd675297da2'),
  'name': 'Cisco',
  'number_of_employees': 63000}]

### 9. Order all the companies by their IPO price in a descending order.

In [178]:
# Your Code
q9 = {"ipo.valuation_amount":{'$not': { '$type': "null" }}}
list(db.companies.find(q9).sort("ipo.valuation_amount",-1).limit(5))

[{'_id': ObjectId('52cdef7e4bab8bd67529a8b4'),
  'name': 'GREE',
  'permalink': 'gree',
  'crunchbase_url': 'http://www.crunchbase.com/company/gree',
  'homepage_url': 'http://www.gree-corp.com',
  'blog_url': '',
  'blog_feed_url': '',
  'twitter_username': 'gree_corp',
  'category_code': 'games_video',
  'number_of_employees': 700,
  'founded_year': 2004,
  'founded_month': 12,
  'founded_day': 7,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'mobile-web, japan, tokyo, social-network, mobile-social-network, mobile-games',
  'alias_list': None,
  'email_address': 'inquiry@gree-corp.com',
  'phone_number': '',
  'description': 'Internet media business,SNS,  free game',
  'created_at': 'Sat Dec 20 16:42:57 UTC 2008',
  'updated_at': 'Tue Jan 01 21:37:04 UTC 2013',
  'overview': '<p>GREE provides Japan&#8217;s leading mobile social network, and is at the forefront of mobile technology. GREE was ranked as Japan&#82

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [195]:
# Your Code
list(db.companies.find(projection={"_id":0,"name":1}).sort("number_of_employees",-1).limit(10))

[{'name': 'Siemens'},
 {'name': 'IBM'},
 {'name': 'Toyota'},
 {'name': 'PayPal'},
 {'name': 'Nippon Telegraph and Telephone Corporation'},
 {'name': 'Samsung Electronics'},
 {'name': 'Accenture'},
 {'name': 'Tata Consultancy Services'},
 {'name': 'Flextronics International'},
 {'name': 'Safeway'}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [198]:
# Your Code
q11 = {"founded_month":{'$gt': 6 }}
list(db.companies.find(q11, {"_id":0,"name":1}).limit(1000))

[{'name': 'Zoho'},
 {'name': 'Wetpaint'},
 {'name': 'Omnidrive'},
 {'name': 'Digg'},
 {'name': 'Joost'},
 {'name': 'Plaxo'},
 {'name': 'eBay'},
 {'name': 'Powerset'},
 {'name': 'Kyte'},
 {'name': 'Thoof'},
 {'name': 'Jingle Networks'},
 {'name': 'Wesabe'},
 {'name': 'LifeLock'},
 {'name': 'SmugMug'},
 {'name': 'Skype'},
 {'name': 'Pando Networks'},
 {'name': 'Ikan'},
 {'name': 'delicious'},
 {'name': 'AllPeers'},
 {'name': 'Wize'},
 {'name': 'SellABand'},
 {'name': 'iContact'},
 {'name': 'MeeVee'},
 {'name': 'blinkx'},
 {'name': 'Zlio'},
 {'name': 'Jaiku'},
 {'name': 'Yelp'},
 {'name': 'Fleck'},
 {'name': 'Yapta'},
 {'name': 'SideStep'},
 {'name': 'RockYou'},
 {'name': 'Instructables'},
 {'name': 'Swivel'},
 {'name': 'Slide'},
 {'name': 'Google'},
 {'name': 'Netvibes'},
 {'name': 'TheFind'},
 {'name': 'Sway'},
 {'name': 'CornerWorld'},
 {'name': 'CrowdVine'},
 {'name': 'FlickIM'},
 {'name': 'Terabitz'},
 {'name': 'Kaboodle'},
 {'name': 'Wink'},
 {'name': 'GigaOM'},
 {'name': 'Go2Web20'

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [ ]:
# Your Code

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [200]:
# Your Code
q12 = {"acquisition.acquired_year":{'$gte': 2010 }}
list(db.companies.find(q12, {"name":1, "acquisition":1}).sort("acquisition.price_amount",-1).limit(5))

[{'_id': ObjectId('52cdef7c4bab8bd675298876'),
  'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'_id': ObjectId('52cdef7f4bab8bd67529c228'),
  'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_comp

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [202]:
# Your Code
list(db.companies.find(projection={"name":1, "founded_year":1}).sort("founded_year",-1).limit(5))

[{'_id': ObjectId('52cdef7c4bab8bd675297fec'),
  'name': 'Fixya',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd67529801f'),
  'name': 'Wamba',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd6752982d4'),
  'name': 'Advaliant',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd67529830a'),
  'name': 'Fluc',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7d4bab8bd675298ea7'),
  'name': 'iBazar',
  'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [203]:
# Your Code
q15 = {"acquisition.acquired_day":{'$lte':7 }}
list(db.companies.find(q15, {"name":1, "founded_year":1}).sort("acquisition.price_amount",-1).limit(10))

[{'_id': ObjectId('52cdef7e4bab8bd67529a2b5'),
  'name': 'National Semiconductor',
  'founded_year': 1959},
 {'_id': ObjectId('52cdef7d4bab8bd67529910e'),
  'name': 'The Weather Channel',
  'founded_year': 1982},
 {'_id': ObjectId('52cdef7c4bab8bd6752986f7'),
  'name': 'Macromedia',
  'founded_year': 1992},
 {'_id': ObjectId('52cdef7f4bab8bd67529c6b0'),
  'name': 'Interactive Data',
  'founded_year': None},
 {'_id': ObjectId('52cdef7e4bab8bd67529aba6'),
  'name': 'Qualcomm Atheros',
  'founded_year': 1998},
 {'_id': ObjectId('52cdef7d4bab8bd675299f6b'),
  'name': 'ExactTarget',
  'founded_year': 2000},
 {'_id': ObjectId('52cdef7e4bab8bd67529a778'),
  'name': 'Quest Software',
  'founded_year': None},
 {'_id': ObjectId('52cdef7d4bab8bd675299f78'),
  'name': 'Legent Corporation',
  'founded_year': None},
 {'_id': ObjectId('52cdef7d4bab8bd675298e3a'),
  'name': 'Digital Insight',
  'founded_year': None},
 {'_id': ObjectId('52cdef7d4bab8bd675299160'),
  'name': 'Equallogic',
  'founded_yea

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [208]:
# Your Code
q16 = {"$and":[{"category_code":"web"},{"number_of_employees":{"$gt":4000}}]}
list(db.companies.find(q16, {"name":1,"number_of_employees":1, "_id":0}).sort("number_of_employees",1).limit(10))

[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group', 'number_of_employees': 10000},
 {'name': 'Groupon', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'number_of_employees': 13600},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Experian', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [211]:
# Your Code
q17 = {"$and":[{"acquisition.price_currency_code":"EUR"},{"acquisition.price_amount":{"$gt":10000000}}]}
list(db.companies.find(q17, {"name":1,"_id":0}).limit(5))

[{'name': 'ZYB'},
 {'name': 'Apertio'},
 {'name': 'Greenfield Online'},
 {'name': 'Webedia'},
 {'name': 'Wayfinder'}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [212]:
# Your Code
q18 = {"acquisition.acquired_month":{"$lte":3}}
list(db.companies.find(q18, {"name":1,"_id":0, "acquisition":1}).limit(10))

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [215]:
# Your Code
q19 = {"$and":[{"acquisition.acquired_year":{"$gte":2011}},{"founded_year":{"$gte":2000}},{"founded_year":{"$lte":2010}}]}
list(db.companies.find(q19, {"name":1,"_id":0}).limit(5))

[{'name': 'Wetpaint'},
 {'name': 'Geni'},
 {'name': 'Digg'},
 {'name': 'Kyte'},
 {'name': 'Jingle Networks'}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code
q19 = {"$and":[{"acquisition.acquired_year":{"$gte":2011}},{"founded_year":{"$gte":2000}},{"founded_year":{"$lte":2010}}]}
list(db.companies.find(q19, {"name":1,"_id":0}).limit(5))